# Installations & Imports & Constants

(Run after the Drive connection is established)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
!pip install -r "/content/drive/My Drive/Startrace_Project_Data/requirements.txt"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 11.2 MB/s eta 0:00:00
  Attempting uninstall: threadpoolctl
    Found existing installation: threadpoolctl 3.5.0
    Uninstalling threadpoolctl-3.5.0:
      Successfully uninstalled threadpoolctl-3.5.0
  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.2
    Uninstall

In [5]:
import pandas as pd
import numpy as np
import pywt
import os
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.fft import fft
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.impute import SimpleImputer

In [8]:
EXOPLANET_DATA_FILE = "/content/drive/My Drive/Startrace_Project_Data/cumulative.csv"

# Data

In [9]:
class ExoplanetData:

    def __init__(self, file_path: str):
        self.file_path = file_path
        self.df = self._load_data()

    def _load_data(self) -> pd.DataFrame:
        df = pd.read_csv(self.file_path, comment='#')
        return df

    def show_head(self, n: int = 5):
        print(self.df.head(n))

    def summary(self) -> pd.DataFrame:
        return self.df.describe()

    def filter_by_disposition(self, disposition: str) -> pd.DataFrame:
        return self.df[self.df['koi_disposition'] == disposition]

    def get_columns(self) -> list:
        return list(self.df.columns)

    def get_dataframe_with_columns(self, columns: list) -> pd.DataFrame:
        existing_columns = [col for col in columns if col in self.df.columns]
        return self.df[existing_columns]

    def extract_statistical_features(self, columns: list) -> pd.DataFrame:
        feature_df = pd.DataFrame(index=self.df.index)
        for col in columns:
            if col in self.df.columns:
                mean_val = self.df[col].mean()
                std_val = self.df[col].std()
                min_val = self.df[col].min()
                max_val = self.df[col].max()
                median_val = self.df[col].median()

                feature_df[f'{col}_mean'] = mean_val
                feature_df[f'{col}_std'] = std_val
                feature_df[f'{col}_min'] = min_val
                feature_df[f'{col}_max'] = max_val
                feature_df[f'{col}_median'] = median_val
        return feature_df

    def extract_fourier_features(self, columns: list) -> pd.DataFrame:
        feature_df = pd.DataFrame(index=self.df.index)
        for col in columns:
            if col in self.df.columns:
                fft_values = np.abs(fft(self.df[col]))
                feature_df[f'{col}_fft_mean'] = np.mean(fft_values)
                feature_df[f'{col}_fft_std'] = np.std(fft_values)
                feature_df[f'{col}_fft_max'] = np.max(fft_values)
        return feature_df

    def extract_wavelet_features(self, columns: list, wavelet='haar') -> pd.DataFrame:
        feature_df = pd.DataFrame(index=self.df.index)
        for col in columns:
            if col in self.df.columns:
                # Her satır için Wavelet dönüşümü uygula
                coeffs = pywt.wavedec(self.df[col], wavelet, level=1)  # 1 seviyeli ayrıştırma
                # Detay katsayılarının istatistiksel özelliklerini al
                feature_df[f'{col}_wavelet_detail_mean'] = np.mean(coeffs[0])
                feature_df[f'{col}_wavelet_detail_std'] = np.std(coeffs[0])
                feature_df[f'{col}_wavelet_approx_mean'] = np.mean(coeffs[1])
                feature_df[f'{col}_wavelet_approx_std'] = np.std(coeffs[1])
        return feature_df

    def extract_manual_features(self) -> pd.DataFrame:
        feature_df = pd.DataFrame(index=self.df.index)

        if 'koi_depth' in self.df.columns and 'koi_period' in self.df.columns:
            feature_df['depth_period_ratio'] = self.df['koi_depth'] / self.df['koi_period']

        if 'koi_steff' in self.df.columns:
            feature_df['koi_steff_squared'] = self.df['koi_steff'] ** 2

        return feature_df


    def show_target_distribution(self, target_column: str):
        if target_column in self.df.columns:
            print(self.df[target_column].value_counts())
        else:
            print(f"Hata: '{target_column}' sütunu bulunamadı.")

    def remove_candidates(self, target_column: str) -> None:
        if target_column in self.df.columns:
            self.df = self.df[self.df[target_column] != 'CANDIDATE']
            print("CANDIDATE etiketli satırlar temizlendi.")
        else:
            print(f"Hata: '{target_column}' sütunu bulunamadı.")


# Model

In [10]:
class ExoplanetModelTrainer:

    def __init__(self, dataframe: pd.DataFrame, target_column: str,
                 use_statistical: bool = True, use_fourier: bool = True,
                 use_wavelet: bool = True, use_manual: bool = True,
                 feature_columns: list = ['koi_period', 'koi_depth', 'koi_impact', 'koi_duration', 'koi_steff', 'koi_srad'],
                 test_size: float = 0.2, validation_size: float = 0.2, random_state: int = 42,
                 hidden_layer_sizes: tuple = (100,), activation: str = 'relu', solver: str = 'adam', learning_rate: str = 'constant',
                 learning_rate_init: float = 0.001, max_iter: int = 300, output_dir: str = "../reports"):

        self.df = dataframe
        self.target_column = target_column
        self.use_statistical = use_statistical
        self.use_fourier = use_fourier
        self.use_wavelet = use_wavelet
        self.use_manual = use_manual
        self.feature_columns = feature_columns
        self.test_size = test_size
        self.validation_size = validation_size
        self.random_state = random_state
        self.hidden_layer_sizes = hidden_layer_sizes
        self.activation = activation
        self.solver = solver
        self.learning_rate = learning_rate
        self.learning_rate_init = learning_rate_init
        self.max_iter = max_iter
        self.output_dir = output_dir
        self.timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        self.run_dir = os.path.join(self.output_dir, self.timestamp)
        os.makedirs(self.run_dir, exist_ok=True)
        self.X_train, self.X_val, self.X_test, self.y_train, self.y_val, self.y_test = self._prepare_data()
        self.scaler = StandardScaler()
        self.X_train_scaled = self.scaler.fit_transform(self.X_train)
        self.X_val_scaled = self.scaler.transform(self.X_val)
        self.X_test_scaled = self.scaler.transform(self.X_test)
        self.model = MLPClassifier(random_state=self.random_state, hidden_layer_sizes=self.hidden_layer_sizes,
                                    activation=self.activation, solver=self.solver, learning_rate=self.learning_rate,
                                    learning_rate_init=self.learning_rate_init, max_iter=self.max_iter)
        self.train_accuracies = []
        self.val_accuracies = []
        self.test_accuracies = []

    def _prepare_data(self) -> tuple:
        X = pd.DataFrame(index=self.df.df.index)

        if self.use_statistical:
            X = X.join(self.df.extract_statistical_features(self.feature_columns))
        if self.use_fourier:
            X = X.join(self.df.extract_fourier_features(self.feature_columns))
        if self.use_wavelet:
            X = X.join(self.df.extract_wavelet_features(self.feature_columns))
        if self.use_manual:
            X = X.join(self.df.extract_manual_features())

        imputer = SimpleImputer(strategy='mean')
        X = imputer.fit_transform(X)

        y = self.df.df[self.target_column]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=self.test_size, random_state=self.random_state, shuffle=False)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=self.validation_size / (1 - self.test_size), random_state=self.random_state, shuffle = False)
        return X_train, X_val, X_test, y_train, y_val, y_test

    def train_model(self):

        for epoch in range(self.max_iter):
            self.model.partial_fit(self.X_train_scaled, self.y_train, classes=pd.unique(self.y_train))  # classes parametresi eklendi

            y_train_pred = self.model.predict(self.X_train_scaled)
            train_accuracy = accuracy_score(self.y_train, y_train_pred)
            self.train_accuracies.append(train_accuracy)

            y_val_pred = self.model.predict(self.X_val_scaled)
            val_accuracy = accuracy_score(self.y_val, y_val_pred)
            self.val_accuracies.append(val_accuracy)

            y_test_pred = self.model.predict(self.X_test_scaled)
            test_accuracy = accuracy_score(self.y_test, y_test_pred)
            self.test_accuracies.append(test_accuracy)

            print(f"Epoch {epoch+1}/{self.max_iter}, Train Accuracy: {train_accuracy:.4f}, Validation Accuracy: {val_accuracy:.4f}, Test Accuracy: {test_accuracy:.4f}")

    def evaluate_train_set(self):
        y_pred = self.model.predict(self.X_train_scaled)
        accuracy = accuracy_score(self.y_train, y_pred)
        report = classification_report(self.y_train, y_pred)
        cm = confusion_matrix(self.y_train, y_pred)
        return accuracy, report, cm

    def evaluate_model(self):
        y_pred = self.model.predict(self.X_test_scaled)
        accuracy = accuracy_score(self.y_test, y_pred)
        report = classification_report(self.y_test, y_pred)
        cm = confusion_matrix(self.y_test, y_pred)
        return accuracy, report, cm

    def evaluate_validation_set(self):
        y_pred = self.model.predict(self.X_val_scaled)
        accuracy = accuracy_score(self.y_val, y_pred)
        report = classification_report(self.y_val, y_pred)
        cm = confusion_matrix(self.y_val, y_pred)
        return accuracy, report, cm

    def save_results(self, test_accuracy: float, test_report: str, test_cm: list, val_accuracy: float, val_report: str, val_cm: list, train_accuracy: float, train_report: str, train_cm: list):         # Metin dosyasına model yapısını ve sonuçları kaydet
        model_summary_path = os.path.join(self.run_dir, "model_summary.txt")
        with open(model_summary_path, "w") as f:
            f.write(f"Model Yapısı:\n")
            f.write(f"Hidden Layer Sizes: {self.hidden_layer_sizes}\n")
            f.write(f"Activation Function: {self.activation}\n")
            f.write(f"Solver: {self.solver}\n")
            f.write(f"Learning Rate: {self.learning_rate}\n")
            f.write(f"Learning Rate Init: {self.learning_rate_init}\n")
            f.write(f"Max Iterations: {self.max_iter}\n")
            f.write(f"\nSon Epoch Train Seti Doğruluğu: {self.train_accuracies[-1]}\n")
            f.write(f"\nSon Epoch Test Seti Doğruluğu: {test_accuracy}\n")
            f.write(f"Test Seti Sınıflandırma Raporu:\n{test_report}\n")
            f.write(f"\nSon Epoch Validation Seti Doğruluğu: {val_accuracy}\n")
            f.write(f"Validation Seti Sınıflandırma Raporu:\n{val_report}\n")
            f.write(f"Test Size: {self.test_size}\n")
            f.write(f"Validation Size: {self.validation_size}\n")
            f.write(f"Random State: {self.random_state}\n")
            f.write(f"\nSon Epoch Train Seti Doğruluğu: {self.train_accuracies[-1]}\n")
            f.write(f"\nSon Epoch Test Seti Doğruluğu: {self.test_accuracies[-1]}\n")

        plt.figure(figsize=(12, 6))
        plt.plot(self.train_accuracies, label='Train Accuracy', color='red')
        plt.plot(self.val_accuracies, label='Validation Accuracy', color='green')
        plt.xlabel('Epoch')
        plt.ylabel('Doğruluk')
        plt.title('Epoch\'a Göre Train ve Validation Doğruluğu')
        plt.legend()
        plt.grid(True)
        accuracy_plot_path = os.path.join(self.run_dir, "accuracy_vs_epoch.png")
        plt.savefig(accuracy_plot_path)
        plt.plot(self.test_accuracies, label="Test Accuracy", color="blue") #test verisi eklendi
        plt.title('Epoch\'a Göre Train, Validation ve Test Doğruluğu')
        plt.close()

        self._plot_confusion_matrix(test_cm, "Test Confusion Matrix", os.path.join(self.run_dir, "test_confusion_matrix.png"))
        self._plot_confusion_matrix(val_cm, "Validation Confusion Matrix", os.path.join(self.run_dir, "validation_confusion_matrix.png"))
        self._plot_confusion_matrix(train_cm, "Train Confusion Matrix", os.path.join(self.run_dir, "train_confusion_matrix.png"))

    def _plot_confusion_matrix(self, cm: list, title: str, filepath: str):
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
        plt.title(title)
        plt.ylabel('Gerçek Etiketler')
        plt.xlabel('Tahmin Edilen Etiketler')
        plt.savefig(filepath)
        plt.close()

    def get_model(self):
        return self.model

    def get_test_data(self):
        return self.X_test_scaled, self.y_test

# Load Data & Train Model

In [12]:
try:
  exoplanet_data = ExoplanetData(EXOPLANET_DATA_FILE)
  target_column = 'koi_disposition'
  exoplanet_data.remove_candidates(target_column)

  #feature_columns = ['koi_period', 'koi_depth', 'koi_impact', 'koi_duration', 'koi_steff', 'koi_srad']

  feature_columns = [
          'koi_period',      # Yörünge periyodu (gezegenin yıldız etrafında dönme süresi)
          'koi_duration',    # Geçiş süresi (gezegenin yıldızın önünden geçme süresi)
          'koi_depth',       # Geçiş derinliği (yıldızın parlaklığındaki azalma miktarı)
          'koi_impact',      # Geçişin etkisi (gezegenin yıldızın merkezinden ne kadar uzakta geçtiği)
          'koi_ror',         # Gezegenin yarıçapının yıldızın yarıçapına oranı
          'koi_teq',         # Gezegenin denge sıcaklığı
          'koi_model_snr',   # Model uyum sinyal-gürültü oranı (geçiş sinyalinin ne kadar belirgin olduğu)
          'koi_steff',       # Yıldızın etkin sıcaklığı
          'koi_slogg',       # Yıldızın yüzey çekimi
          'koi_srad'         # Yıldızın yarıçapı
  ]

  data = exoplanet_data.get_dataframe_with_columns([target_column] + feature_columns)
  data = data.dropna()

  exoplanet_data.show_target_distribution(target_column)

  data[target_column] = data[target_column].astype('category').cat.codes

  trainer = ExoplanetModelTrainer(dataframe=exoplanet_data,
                                  target_column=target_column,
                                  use_statistical=True,
                                  use_fourier=True,
                                  use_wavelet=True,
                                  use_manual=True,
                                  feature_columns=feature_columns,
                                  test_size=0.2, validation_size=0.2, random_state=42,
                                  hidden_layer_sizes=(128, 64),
                                  activation='relu',
                                  solver='adam',
                                  learning_rate='adaptive',
                                  learning_rate_init=0.001,
                                  max_iter=50,
                                  output_dir="/content/drive/My Drive/Startrace_Project_Data/logs")

  trainer.train_model()

  accuracy, report, test_cm = trainer.evaluate_model()
  val_accuracy, val_report, val_cm = trainer.evaluate_validation_set()
  train_accuracy, train_report, train_cm = trainer.evaluate_train_set()

  trainer.save_results(accuracy, report, test_cm, val_accuracy, val_report, val_cm, train_accuracy, train_report, train_cm)

  print(f"Model Doğruluğu (Train Seti): {train_accuracy}")
  print(f"Model Doğruluğu (Test Seti): {accuracy}")
  print("Sınıflandırma Raporu (Test Seti):\n", report)
  print(f"Model Doğruluğu (Validation Seti): {val_accuracy}")
  print("Sınıflandırma Raporu (Validation Seti):\n", val_report)
  print(f"Raporlar {trainer.run_dir} dizinine kaydedildi.")

except FileNotFoundError:
  print(f"{EXOPLANET_DATA_FILE} dosyası bulunamadı.")
except KeyError as e:
  print(f"Sütun hatası: {e}. Lütfen sütun adlarını kontrol edin.")
except Exception as e:
  print(f"Beklenmedik bir hata oluştu: {e}")

CANDIDATE etiketli satırlar temizlendi.
koi_disposition
FALSE POSITIVE    4839
CONFIRMED         2743
Name: count, dtype: int64


/usr/local/lib/python3.11/dist-packages/sklearn/impute/_base.py:635: UserWarning: Skipping features without any observed values: ['koi_depth_fft_mean' 'koi_depth_fft_std' 'koi_depth_fft_max'
 'koi_impact_fft_mean' 'koi_impact_fft_std' 'koi_impact_fft_max'
 'koi_ror_fft_mean' 'koi_ror_fft_std' 'koi_ror_fft_max' 'koi_teq_fft_mean'
 'koi_teq_fft_std' 'koi_teq_fft_max' 'koi_model_snr_fft_mean'
 'koi_model_snr_fft_std' 'koi_model_snr_fft_max' 'koi_steff_fft_mean'
 'koi_steff_fft_std' 'koi_steff_fft_max' 'koi_slogg_fft_mean'
 'koi_slogg_fft_std' 'koi_slogg_fft_max' 'koi_srad_fft_mean'
 'koi_srad_fft_std' 'koi_srad_fft_max' 'koi_depth_wavelet_detail_mean'
 'koi_depth_wavelet_detail_std' 'koi_depth_wavelet_approx_mean'
 'koi_depth_wavelet_approx_std' 'koi_impact_wavelet_detail_mean'
 'koi_impact_wavelet_detail_std' 'koi_impact_wavelet_approx_mean'
 'koi_impact_wavelet_approx_std' 'koi_ror_wavelet_detail_mean'
 'koi_ror_wavelet_detail_std' 'koi_ror_wavelet_approx_mean'
 'koi_ror_wavelet_approx_

Epoch 1/50, Train Accuracy: 0.6401, Validation Accuracy: 0.4199, Test Accuracy: 0.5208
Epoch 2/50, Train Accuracy: 0.6489, Validation Accuracy: 0.3856, Test Accuracy: 0.4964
Epoch 3/50, Train Accuracy: 0.6645, Validation Accuracy: 0.4904, Test Accuracy: 0.5557
Epoch 4/50, Train Accuracy: 0.6689, Validation Accuracy: 0.4957, Test Accuracy: 0.5636
Epoch 5/50, Train Accuracy: 0.6744, Validation Accuracy: 0.4911, Test Accuracy: 0.5702
Epoch 6/50, Train Accuracy: 0.6774, Validation Accuracy: 0.4852, Test Accuracy: 0.5669
Epoch 7/50, Train Accuracy: 0.6794, Validation Accuracy: 0.4832, Test Accuracy: 0.5669
Epoch 8/50, Train Accuracy: 0.6814, Validation Accuracy: 0.4871, Test Accuracy: 0.5689
Epoch 9/50, Train Accuracy: 0.6847, Validation Accuracy: 0.4878, Test Accuracy: 0.5735
Epoch 10/50, Train Accuracy: 0.6867, Validation Accuracy: 0.4951, Test Accuracy: 0.5768
Epoch 11/50, Train Accuracy: 0.6887, Validation Accuracy: 0.4970, Test Accuracy: 0.5808
Epoch 12/50, Train Accuracy: 0.6895, Vali